In [161]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import statsmodels.api as sm

from matplotlib import font_manager, rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
import os

path = "/content/drive/Shared drives/경제정보분석"
path2 = "/content/drive/My Drive/2020 가을학기/경정분/project"
os.chdir(path2)

os.getcwd()

'/content/drive/My Drive/2020 가을학기/경정분/project'

In [127]:
# 카드 데이터 불러오기
card = pd.read_csv(path + "/KT_data_20200717/card_20200717.csv")
card.tail()

,receipt_dttm,adstrd_code,adstrd_nm,mrhst_induty_cl_code,mrhst_induty_cl_nm,selng_cascnt,salamt
3713947,20200614,1121586000,구의2동,4004,대형할인점,136,1195640
3713948,20200614,1121584700,자양4동,9010,인테 리어,2,13100
3713949,20200614,1162052500,보라매동,6140,주 차 장,2,14500
3713950,20200614,1135056000,월계1동,9998,기타전문점,1,38000
3713951,20200614,1159060500,흑석동,9204,사무통신기기수리,1,20000


In [128]:
# 코로나 데이터 불러오기
covid = pd.read_csv(path2 + "/card_sum_covid_cum.csv", usecols = ["signgu_nm","date", "count", "cumsum"])
covid

,signgu_nm,date,count,cumsum
0,강남구,2020-01-04,0,0
1,강동구,2020-01-04,0,0
2,강북구,2020-01-04,0,0
3,강서구,2020-01-04,0,0
4,관악구,2020-01-04,0,0
...,...,...,...,...
3870,용산구,2020-06-14,1,46
3871,은평구,2020-06-14,1,43
3872,종로구,2020-06-14,0,21
3873,중구,2020-06-14,0,10


In [129]:
covid["date"] = pd.to_datetime(covid["date"])

In [130]:
# 유동인구수 데이터 불러오기
pop_merge = pd.read_csv(path2 + '/temppop.csv')
pop_merge.tail()

,Unnamed: 0,signgu_nm,date,popltn_cascnt
4195,4195,중랑구,2020-06-12,1724648
4196,4196,중랑구,2020-06-13,1689244
4197,4197,중랑구,2020-06-14,1436522
4198,4198,중랑구,2020-06-15,1663686
4199,4199,중랑구,2020-06-16,1651099


In [131]:
pop_merge["date"] = pd.to_datetime(pop_merge["date"])

## Fashion & Covid & Pop DataFrame 정리하기

In [132]:
admin = pd.read_csv(path + "/KT_data_20200717/adstrd_master.csv")
admin = admin[admin.brtc_nm == "서울특별시"]
admin['adstrd_code'] = admin.adstrd_code.apply(int)
admin.describe()
admin.adstrd_code  = admin.adstrd_code * 100
admin

,adstrd_code,adstrd_nm,brtc_nm,signgu_nm
0,1111051500,청운효자동,서울특별시,종로구
1,1111053000,사직동,서울특별시,종로구
2,1111054000,삼청동,서울특별시,종로구
3,1111055000,부암동,서울특별시,종로구
4,1111056000,평창동,서울특별시,종로구
...,...,...,...,...
420,1174065000,성내2동,서울특별시,강동구
421,1174066000,성내3동,서울특별시,강동구
422,1174068500,길동,서울특별시,강동구
423,1174069000,둔촌1동,서울특별시,강동구


In [133]:
lst = ['정장', '악세사리', '신발', '내의판매점', '기타의류', '가방', '아동의류', '단체복', '와이셔츠/타이', '캐쥬얼의류']
fashion = card[card.mrhst_induty_cl_nm.isin(lst)]
fashion.head()

,receipt_dttm,adstrd_code,adstrd_nm,mrhst_induty_cl_code,mrhst_induty_cl_nm,selng_cascnt,salamt
8,20200104,1174066000,성내3동,4201,정장,13,743000
80,20200104,1174062000,천호3동,4201,정장,40,2823850
113,20200104,1174061000,천호2동,4201,정장,38,1501300
138,20200104,1174060000,천호1동,4201,정장,22,908000
188,20200104,1174057000,암사1동,4201,정장,44,3503390


In [134]:
fashion = pd.merge(fashion, admin, on = 'adstrd_code', how = 'left')

fashion.head()

,receipt_dttm,adstrd_code,adstrd_nm_x,mrhst_induty_cl_code,mrhst_induty_cl_nm,selng_cascnt,salamt,adstrd_nm_y,brtc_nm,signgu_nm
0,20200104,1174066000,성내3동,4201,정장,13,743000,성내3동,서울특별시,강동구
1,20200104,1174062000,천호3동,4201,정장,40,2823850,천호3동,서울특별시,강동구
2,20200104,1174061000,천호2동,4201,정장,38,1501300,천호2동,서울특별시,강동구
3,20200104,1174060000,천호1동,4201,정장,22,908000,천호1동,서울특별시,강동구
4,20200104,1174057000,암사1동,4201,정장,44,3503390,암사1동,서울특별시,강동구


In [135]:
fashion.signgu_nm.unique()

array(['강동구', '송파구', '강남구', '서초구', '관악구', '동작구', '영등포구', '금천구', '구로구',
       '강서구', '양천구', '마포구', '서대문구', '은평구', '노원구', '도봉구', '강북구', '성북구',
       '중랑구', '동대문구', '광진구', '성동구', '용산구', '중구', '종로구'], dtype=object)

In [136]:
fashion_ad = fashion.groupby(['receipt_dttm', 'signgu_nm'])
fashion_ad = fashion_ad.agg(['sum', 'mean'])
fashion_ad = fashion_ad.salamt
fashion_ad = fashion_ad.reset_index(level = [0,1])
fashion_ad = fashion_ad.rename(columns = {'receipt_dttm' : "date", 'signgu_nm_y' : 'signgu_nm'})
fashion_ad["date"] = fashion_ad["date"].apply(str)
fashion_ad['date'] = pd.to_datetime(fashion_ad['date'])

fashion = fashion_ad.set_index('date')

In [137]:
fashion

,signgu_nm,sum,mean
date,,,
2020-01-04,강남구,492452026,2.591853e+07
2020-01-04,강동구,13450740,1.681342e+06
2020-01-04,강북구,3752350,1.876175e+06
2020-01-04,강서구,18507452,1.682496e+06
2020-01-04,관악구,23524490,2.352449e+06
...,...,...,...
2020-06-14,용산구,57899500,2.067839e+06
2020-06-14,은평구,33578800,8.836526e+05
2020-06-14,종로구,62919956,2.169654e+06


In [138]:
fashion_covid = pd.merge(left=fashion, right=covid, how = 'left', on=["signgu_nm", "date"], sort=False)
fashion_covid

,signgu_nm,date,sum,mean,count,cumsum
0,강남구,2020-01-04,492452026,2.591853e+07,0,0
1,강동구,2020-01-04,13450740,1.681342e+06,0,0
2,강북구,2020-01-04,3752350,1.876175e+06,0,0
3,강서구,2020-01-04,18507452,1.682496e+06,0,0
4,관악구,2020-01-04,23524490,2.352449e+06,0,0
...,...,...,...,...,...,...
3870,용산구,2020-06-14,57899500,2.067839e+06,1,46
3871,은평구,2020-06-14,33578800,8.836526e+05,1,43
3872,종로구,2020-06-14,62919956,2.169654e+06,0,21
3873,중구,2020-06-14,431806445,1.053186e+07,0,10


In [139]:
fa_cov_pop = pd.merge(left=fashion_covid, right=pop_merge, how = 'left', on=["signgu_nm", "date"], sort=False)
fa_cov_pop

,signgu_nm,date,sum,mean,count,cumsum,Unnamed: 0,popltn_cascnt
0,강남구,2020-01-04,492452026,2.591853e+07,0,0,3,4268654
1,강동구,2020-01-04,13450740,1.681342e+06,0,0,171,2425402
2,강북구,2020-01-04,3752350,1.876175e+06,0,0,339,1036862
3,강서구,2020-01-04,18507452,1.682496e+06,0,0,507,2465692
4,관악구,2020-01-04,23524490,2.352449e+06,0,0,675,1977717
...,...,...,...,...,...,...,...,...
3870,용산구,2020-06-14,57899500,2.067839e+06,1,46,3525,1848525
3871,은평구,2020-06-14,33578800,8.836526e+05,1,43,3693,1436435
3872,종로구,2020-06-14,62919956,2.169654e+06,0,21,3861,1269669
3873,중구,2020-06-14,431806445,1.053186e+07,0,10,4029,1166980


In [140]:
fa_cov_pop =  fa_cov_pop.set_index(["signgu_nm", "date"])
fa_cov_pop.drop(columns = {"Unnamed: 0"}, inplace = True)
fa_cov_pop.head()
# fa_cov_pop.to_csv("fa_cov_pop.csv")

,,sum,mean,count,cumsum,popltn_cascnt
signgu_nm,date,,,,,
강남구,2020-01-04,492452026,2.591853e+07,0,0,4268654
강동구,2020-01-04,13450740,1.681342e+06,0,0,2425402
강북구,2020-01-04,3752350,1.876175e+06,0,0,1036862
강서구,2020-01-04,18507452,1.682496e+06,0,0,2465692
관악구,2020-01-04,23524490,2.352449e+06,0,0,1977717


In [141]:
# fa_cov_pop["sum"] = np.exp(fa_cov_pop["sum"])
# fa_cov_pop["popltn_cascnt"] = np.exp(fa_cov_pop["popltn_cascnt"])

In [142]:
temp = fa_cov_pop.sort_values(by = ["signgu_nm", "date"])
temp.loc[:,"count_reg1"] = temp['count'].shift(1)
temp.loc[:,"cumsum_reg1"] = temp['cumsum'].shift(1)

temp

sum          mean  ...  count_reg1  cumsum_reg1
signgu_nm date                                 ...                         
강남구       2020-01-04  492452026  2.591853e+07  ...         NaN          NaN
          2020-01-05  628419156  3.307469e+07  ...         0.0          0.0
          2020-01-06  554314760  3.959391e+07  ...         0.0          0.0
          2020-01-07  273889480  1.369447e+07  ...         0.0          0.0
          2020-01-08  342010218  1.628620e+07  ...         0.0          0.0
...                         ...           ...  ...         ...          ...
중랑구       2020-06-10   27009620  6.281307e+05  ...         0.0         21.0
          2020-06-11   22580800  6.102919e+05  ...         0.0         21.0
          2020-06-12   25837300  6.008674e+05  ...         2.0         23.0
          2020-06-13   25077520  5.831981e+05  ...         3.0         26.0
          2020-06-14   20783235  4.948389e+05  ...         1.0         27.0

[3875 rows x 7 columns]

In [143]:
temp = temp.reset_index([0,1])

In [144]:
temp.loc[(temp.date == "2020-01-04"),["count_reg1","cumsum_reg1"]] = np.nan
temp[temp["date"] == "2020-01-04"]

,signgu_nm,date,sum,mean,count,cumsum,popltn_cascnt,count_reg1,cumsum_reg1
0,강남구,2020-01-04,492452026,2.591853e+07,0,0,4268654,NaN,NaN
155,강동구,2020-01-04,13450740,1.681342e+06,0,0,2425402,NaN,NaN
310,강북구,2020-01-04,3752350,1.876175e+06,0,0,1036862,NaN,NaN
465,강서구,2020-01-04,18507452,1.682496e+06,0,0,2465692,NaN,NaN
620,관악구,2020-01-04,23524490,2.352449e+06,0,0,1977717,NaN,NaN
775,광진구,2020-01-04,40343170,3.667561e+06,0,0,2188255,NaN,NaN
930,구로구,2020-01-04,24591860,3.513123e+06,0,0,2017911,NaN,NaN
1085,금천구,2020-01-04,17982960,4.495740e+06,0,0,1067463,NaN,NaN
1240,노원구,2020-01-04,11112900,1.587557e+06,0,0,2165001,NaN,NaN
1395,도봉구,2020-01-04,4073000,5.818571e+05,0,0,1214028,NaN,NaN


In [145]:
fa_cov_pop = temp.set_index(["signgu_nm","date"])

In [146]:
fa_cov_pop

sum          mean  ...  count_reg1  cumsum_reg1
signgu_nm date                                 ...                         
강남구       2020-01-04  492452026  2.591853e+07  ...         NaN          NaN
          2020-01-05  628419156  3.307469e+07  ...         0.0          0.0
          2020-01-06  554314760  3.959391e+07  ...         0.0          0.0
          2020-01-07  273889480  1.369447e+07  ...         0.0          0.0
          2020-01-08  342010218  1.628620e+07  ...         0.0          0.0
...                         ...           ...  ...         ...          ...
중랑구       2020-06-10   27009620  6.281307e+05  ...         0.0         21.0
          2020-06-11   22580800  6.102919e+05  ...         0.0         21.0
          2020-06-12   25837300  6.008674e+05  ...         2.0         23.0
          2020-06-13   25077520  5.831981e+05  ...         3.0         26.0
          2020-06-14   20783235  4.948389e+05  ...         1.0         27.0

[3875 rows x 7 columns]

## Card & Covid & Pop DataFrame 정리하기

In [147]:
admin = pd.read_csv(path + "/KT_data_20200717/adstrd_master.csv")
admin = admin[admin.brtc_nm == "서울특별시"]
admin['adstrd_code'] = admin.adstrd_code.apply(int)
admin.describe()
admin.adstrd_code  = admin.adstrd_code * 100
admin

,adstrd_code,adstrd_nm,brtc_nm,signgu_nm
0,1111051500,청운효자동,서울특별시,종로구
1,1111053000,사직동,서울특별시,종로구
2,1111054000,삼청동,서울특별시,종로구
3,1111055000,부암동,서울특별시,종로구
4,1111056000,평창동,서울특별시,종로구
...,...,...,...,...
420,1174065000,성내2동,서울특별시,강동구
421,1174066000,성내3동,서울특별시,강동구
422,1174068500,길동,서울특별시,강동구
423,1174069000,둔촌1동,서울특별시,강동구


In [148]:
card = pd.merge(card, admin, on = 'adstrd_code', how = 'left')

card.head() 

,receipt_dttm,adstrd_code,adstrd_nm_x,mrhst_induty_cl_code,mrhst_induty_cl_nm,selng_cascnt,salamt,adstrd_nm_y,brtc_nm,signgu_nm
0,20200104,1174066000,성내3동,7041,약국,463,5843230,성내3동,서울특별시,강동구
1,20200104,1174066000,성내3동,7022,치과의원,33,7835550,성내3동,서울특별시,강동구
2,20200104,1174066000,성내3동,7021,한의원,53,4589800,성내3동,서울특별시,강동구
3,20200104,1174066000,성내3동,7020,의원,339,9267240,성내3동,서울특별시,강동구
4,20200104,1174066000,성내3동,6110,자동차정비,19,4441000,성내3동,서울특별시,강동구


In [149]:
temp = card.groupby(['receipt_dttm', 'signgu_nm'])
temp = temp.agg(['sum', 'mean'])
temp = temp.salamt
temp = temp.reset_index(level = [0,1])
temp = temp.rename(columns = {'receipt_dttm' : "date", 'signgu_nm_y' : 'signgu_nm'})
temp["date"] = temp["date"].apply(str)
temp['date'] = pd.to_datetime(temp['date'])

card = temp.set_index('date')

In [150]:
card_covid = pd.merge(left=card, right=covid, how = 'left', on=["signgu_nm", "date"], sort=False)
card_covid

,signgu_nm,date,sum,mean,count,cumsum
0,강남구,2020-01-04,31015754383,4.380756e+07,0,0
1,강동구,2020-01-04,1766901445,4.921731e+06,0,0
2,강북구,2020-01-04,491339680,3.169933e+06,0,0
3,강서구,2020-01-04,7286320793,1.573719e+07,0,0
4,관악구,2020-01-04,2094140059,5.021919e+06,0,0
...,...,...,...,...,...,...
3870,용산구,2020-06-14,6216508795,7.000573e+06,1,46
3871,은평구,2020-06-14,2011764106,2.032085e+06,1,43
3872,종로구,2020-06-14,2528669883,2.775708e+06,0,21
3873,중구,2020-06-14,8324728479,8.999706e+06,0,10


In [151]:
card_cov_pop = pd.merge(left=card_covid, right=pop_merge, how = 'left', on=["signgu_nm", "date"], sort=False)
card_cov_pop

,signgu_nm,date,sum,mean,count,cumsum,Unnamed: 0,popltn_cascnt
0,강남구,2020-01-04,31015754383,4.380756e+07,0,0,3,4268654
1,강동구,2020-01-04,1766901445,4.921731e+06,0,0,171,2425402
2,강북구,2020-01-04,491339680,3.169933e+06,0,0,339,1036862
3,강서구,2020-01-04,7286320793,1.573719e+07,0,0,507,2465692
4,관악구,2020-01-04,2094140059,5.021919e+06,0,0,675,1977717
...,...,...,...,...,...,...,...,...
3870,용산구,2020-06-14,6216508795,7.000573e+06,1,46,3525,1848525
3871,은평구,2020-06-14,2011764106,2.032085e+06,1,43,3693,1436435
3872,종로구,2020-06-14,2528669883,2.775708e+06,0,21,3861,1269669
3873,중구,2020-06-14,8324728479,8.999706e+06,0,10,4029,1166980


In [152]:
card_cov_pop =  card_cov_pop.set_index(["signgu_nm", "date"])
card_cov_pop.drop(columns = {"Unnamed: 0"}, inplace = True)
card_cov_pop.head()

,,sum,mean,count,cumsum,popltn_cascnt
signgu_nm,date,,,,,
강남구,2020-01-04,31015754383,4.380756e+07,0,0,4268654
강동구,2020-01-04,1766901445,4.921731e+06,0,0,2425402
강북구,2020-01-04,491339680,3.169933e+06,0,0,1036862
강서구,2020-01-04,7286320793,1.573719e+07,0,0,2465692
관악구,2020-01-04,2094140059,5.021919e+06,0,0,1977717


In [153]:
temp = card_cov_pop.sort_values(by = ["signgu_nm", "date"])
temp.loc[:,"count_reg1"] = temp['count'].shift(1)
temp.loc[:,"cumsum_reg1"] = temp['cumsum'].shift(1)

temp

sum          mean  ...  count_reg1  cumsum_reg1
signgu_nm date                                   ...                         
강남구       2020-01-04  31015754383  4.380756e+07  ...         NaN          NaN
          2020-01-05  11149011881  1.798228e+07  ...         0.0          0.0
          2020-01-06  12034811932  2.971559e+07  ...         0.0          0.0
          2020-01-07  34435546507  5.056615e+07  ...         0.0          0.0
          2020-01-08  32874537077  4.848752e+07  ...         0.0          0.0
...                           ...           ...  ...         ...          ...
중랑구       2020-06-10   1563681665  1.460020e+06  ...         0.0         21.0
          2020-06-11   1458262768  1.407590e+06  ...         0.0         21.0
          2020-06-12   1479523802  1.405056e+06  ...         2.0         23.0
          2020-06-13   1694347307  1.616744e+06  ...         3.0         26.0
          2020-06-14   1582840231  1.623426e+06  ...         1.0         27.0

[3875 rows x 7 columns]

In [154]:
temp = temp.reset_index([0,1])
temp.loc[(temp.date == "2020-01-04"),["count_reg1","cumsum_reg1"]] = np.nan
temp[temp["date"] == "2020-01-04"]

,signgu_nm,date,sum,mean,count,cumsum,popltn_cascnt,count_reg1,cumsum_reg1
0,강남구,2020-01-04,31015754383,4.380756e+07,0,0,4268654,NaN,NaN
155,강동구,2020-01-04,1766901445,4.921731e+06,0,0,2425402,NaN,NaN
310,강북구,2020-01-04,491339680,3.169933e+06,0,0,1036862,NaN,NaN
465,강서구,2020-01-04,7286320793,1.573719e+07,0,0,2465692,NaN,NaN
620,관악구,2020-01-04,2094140059,5.021919e+06,0,0,1977717,NaN,NaN
775,광진구,2020-01-04,1494472816,4.542471e+06,0,0,2188255,NaN,NaN
930,구로구,2020-01-04,10842506695,3.054227e+07,0,0,2017911,NaN,NaN
1085,금천구,2020-01-04,1448054788,6.830447e+06,0,0,1067463,NaN,NaN
1240,노원구,2020-01-04,1468625449,3.990830e+06,0,0,2165001,NaN,NaN
1395,도봉구,2020-01-04,640561501,2.614537e+06,0,0,1214028,NaN,NaN


In [155]:
card_cov_pop = temp.set_index(["signgu_nm","date"])
card_cov_pop

sum          mean  ...  count_reg1  cumsum_reg1
signgu_nm date                                   ...                         
강남구       2020-01-04  31015754383  4.380756e+07  ...         NaN          NaN
          2020-01-05  11149011881  1.798228e+07  ...         0.0          0.0
          2020-01-06  12034811932  2.971559e+07  ...         0.0          0.0
          2020-01-07  34435546507  5.056615e+07  ...         0.0          0.0
          2020-01-08  32874537077  4.848752e+07  ...         0.0          0.0
...                           ...           ...  ...         ...          ...
중랑구       2020-06-10   1563681665  1.460020e+06  ...         0.0         21.0
          2020-06-11   1458262768  1.407590e+06  ...         0.0         21.0
          2020-06-12   1479523802  1.405056e+06  ...         2.0         23.0
          2020-06-13   1694347307  1.616744e+06  ...         3.0         26.0
          2020-06-14   1582840231  1.623426e+06  ...         1.0         27.0

[3875 rows x 7 columns]

# Fashion Reg

## Pooled OLS

In [ ]:
!pip install linearmodels

In [158]:
# Pooled OLS with reg1 cov
from linearmodels.panel import PooledOLS
import statsmodels.api as sm

exog_vars = ['cumsum_reg1','popltn_cascnt']
exog = sm.add_constant(fa_cov_pop[exog_vars])
mod = PooledOLS(fa_cov_pop["sum"], exog)
pooled_res = mod.fit(missing = "drop")
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:                    sum   R-squared:                        0.2652
Estimator:                  PooledOLS   R-squared (Between):              0.3442
No. Observations:                3850   R-squared (Within):              -0.1825
Date:                Tue, Dec 15 2020   R-squared (Overall):              0.2652
Time:                        10:22:59   Log-likelihood                -7.484e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      694.15
Entities:                          25   P-value                           0.0000
Avg Obs:                       154.00   Distribution:                  F(2,3847)
Min Obs:                       154.00                                           
Max Obs:                       154.00   F-statistic (robust):             694.15
                            

/usr/local/lib/python3.6/dist-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [163]:
#mean
# Pooled OLS with reg1 cov
from linearmodels.panel import PooledOLS
import statsmodels.api as sm

exog_vars = ['cumsum_reg1','popltn_cascnt']
exog = sm.add_constant(fa_cov_pop[exog_vars])
mod = PooledOLS(fa_cov_pop["mean"], exog)
pooled_res = mod.fit(missing = "drop")
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:                   mean   R-squared:                        0.1666
Estimator:                  PooledOLS   R-squared (Between):              0.2948
No. Observations:                3850   R-squared (Within):               0.0456
Date:                Tue, Dec 15 2020   R-squared (Overall):              0.1666
Time:                        11:41:58   Log-likelihood                 -6.23e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      384.40
Entities:                          25   P-value                           0.0000
Avg Obs:                       154.00   Distribution:                  F(2,3847)
Min Obs:                       154.00                                           
Max Obs:                       154.00   F-statistic (robust):             384.40
                            

/usr/local/lib/python3.6/dist-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [162]:
#mean
# Pooled OLS with reg1 cov
from linearmodels.panel import PooledOLS
import statsmodels.api as sm

exog_vars = ['count_reg1','popltn_cascnt']
exog = sm.add_constant(fa_cov_pop[exog_vars])
mod = PooledOLS(fa_cov_pop["mean"], exog)
pooled_res = mod.fit(missing = "drop")
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:                   mean   R-squared:                        0.1406
Estimator:                  PooledOLS   R-squared (Between):              0.2810
No. Observations:                3850   R-squared (Within):               0.0082
Date:                Tue, Dec 15 2020   R-squared (Overall):              0.1406
Time:                        11:41:52   Log-likelihood                -6.235e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      314.59
Entities:                          25   P-value                           0.0000
Avg Obs:                       154.00   Distribution:                  F(2,3847)
Min Obs:                       154.00                                           
Max Obs:                       154.00   F-statistic (robust):             314.59
                            

/usr/local/lib/python3.6/dist-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [164]:
# from statsmodels.compat import lzip
# import statsmodels.stats.api as sms

# #perform Bresuch-Pagan test
# names = ['Lagrange multiplier statistic', 'p-value',
#         'f-value', 'f p-value']
# test = sms.het_breuschpagan(pooled_res.resid, pooled_res.model.exog)

# lzip(names, test)

# [('Lagrange multiplier statistic', 6.003951995818433),
#  ('p-value', 0.11141811013399583),
#  ('f-value', 3.004944880309618),
#  ('f p-value', 0.11663863538255281)]

## Random Effects

In [167]:
from linearmodels.panel import RandomEffects

exog_vars = ['cumsum_reg1','popltn_cascnt']
mod = RandomEffects(fa_cov_pop["mean"], fa_cov_pop[exog_vars])
re_res = mod.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:                   mean   R-squared:                        0.0589
Estimator:              RandomEffects   R-squared (Between):              0.5599
No. Observations:                3850   R-squared (Within):               0.0520
Date:                Tue, Dec 15 2020   R-squared (Overall):              0.3680
Time:                        11:43:47   Log-likelihood                -6.128e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      120.47
Entities:                          25   P-value                           0.0000
Avg Obs:                       154.00   Distribution:                  F(2,3848)
Min Obs:                       154.00                                           
Max Obs:                       154.00   F-statistic (robust):             120.47
                            

/usr/local/lib/python3.6/dist-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [168]:
re_res.variance_decomposition

Effects                   2.995860e+12
Residual                  3.909834e+12
Percent due to Effects    4.338246e-01
Name: Variance Decomposition, dtype: float64

In [169]:
re_res.theta.head()

,theta
signgu_nm,
강남구,0.90833
강동구,0.90833
강북구,0.90833
강서구,0.90833
관악구,0.90833


## BetweenOLS

In [100]:
from linearmodels.panel import BetweenOLS

exog_vars = ['cumsum_reg1','popltn_cascnt']
exog = sm.add_constant(fa_cov_pop[exog_vars])
mod = BetweenOLS(fa_cov_pop["sum"], exog)
be_res = mod.fit()
print(be_res)

                         BetweenOLS Estimation Summary                          
Dep. Variable:                    sum   R-squared:                        0.3558
Estimator:                 BetweenOLS   R-squared (Between):              0.3558
No. Observations:                  25   R-squared (Within):              -0.6172
Date:                Tue, Dec 15 2020   R-squared (Overall):              0.2099
Time:                        07:14:54   Log-likelihood                   -482.30
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      6.0766
Entities:                          25   P-value                           0.0079
Avg Obs:                       154.00   Distribution:                    F(2,22)
Min Obs:                       154.00                                           
Max Obs:                       154.00   F-statistic (robust):             6.0766
                            

/usr/local/lib/python3.6/dist-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


## PanelOLS   
; fixed effect by entity_effects = True

In [170]:
from linearmodels.panel import PanelOLS

exog_vars = ['cumsum_reg1','popltn_cascnt']
exog = sm.add_constant(fa_cov_pop[exog_vars])
mod = PanelOLS(fa_cov_pop["mean"], exog, entity_effects = True)
fe_res = mod.fit()
print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   mean   R-squared:                        0.0521
Estimator:                   PanelOLS   R-squared (Between):              0.2298
No. Observations:                3850   R-squared (Within):               0.0521
Date:                Tue, Dec 15 2020   R-squared (Overall):              0.1384
Time:                        11:44:27   Log-likelihood                -6.126e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      105.16
Entities:                          25   P-value                           0.0000
Avg Obs:                       154.00   Distribution:                  F(2,3823)
Min Obs:                       154.00                                           
Max Obs:                       154.00   F-statistic (robust):             105.16
                            

/usr/local/lib/python3.6/dist-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [171]:
from linearmodels.panel import PanelOLS

exog_vars = ['cumsum_reg1','popltn_cascnt']
exog = sm.add_constant(fa_cov_pop[exog_vars])
mod = PanelOLS(fa_cov_pop["mean"], exog, entity_effects = True, time_effects = True)
te_fe_res = mod.fit()
print(te_fe_res)

/usr/local/lib/python3.6/dist-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          PanelOLS Estimation Summary                           
Dep. Variable:                   mean   R-squared:                        0.0148
Estimator:                   PanelOLS   R-squared (Between):             -0.0208
No. Observations:                3850   R-squared (Within):               0.0412
Date:                Tue, Dec 15 2020   R-squared (Overall):              0.0111
Time:                        11:45:14   Log-likelihood                -6.056e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      27.573
Entities:                          25   P-value                           0.0000
Avg Obs:                       154.00   Distribution:                  F(2,3670)
Min Obs:                       154.00                                           
Max Obs:                       154.00   F-statistic (robust):             27.573
                            

In [172]:
from linearmodels.panel import compare
print(compare({"BE" : be_res, "RE" : re_res, "Pooled" : pooled_res}))

                        Model Comparison                       
                                BE              RE       Pooled
---------------------------------------------------------------
Dep. Variable                  sum            mean         mean
Estimator               BetweenOLS   RandomEffects    PooledOLS
No. Observations                25            3850         3850
Cov. Est.               Unadjusted      Unadjusted   Unadjusted
R-squared                   0.3558          0.0589       0.1666
R-Squared (Within)         -0.6172          0.0520       0.0456
R-Squared (Between)         0.3558          0.5599       0.2948
R-Squared (Overall)         0.2099          0.3680       0.1666
F-statistic                 6.0766          120.47       384.40
P-value (F-stat)            0.0079          0.0000       0.0000
===================== ============ =============== ============
const                    -5.68e+07                   -6.918e+05
                         (-1.6622)      

# Card Reg

## Pooled OLS

In [177]:
# Pooled OLS with reg1 cov
from linearmodels.panel import PooledOLS
import statsmodels.api as sm

exog_vars = ['count_reg1','popltn_cascnt']
exog = sm.add_constant(card_cov_pop[exog_vars])
mod = PooledOLS(card_cov_pop["sum"], exog)
pooled_res = mod.fit()
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:                    sum   R-squared:                        0.2041
Estimator:                  PooledOLS   R-squared (Between):              0.5283
No. Observations:                3850   R-squared (Within):               0.0325
Date:                Tue, Dec 15 2020   R-squared (Overall):              0.2041
Time:                        11:53:23   Log-likelihood                -9.633e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      493.31
Entities:                          25   P-value                           0.0000
Avg Obs:                       154.00   Distribution:                  F(2,3847)
Min Obs:                       154.00                                           
Max Obs:                       154.00   F-statistic (robust):             493.31
                            

/usr/local/lib/python3.6/dist-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [178]:
# Pooled OLS with cumsum
from linearmodels.panel import PooledOLS
import statsmodels.api as sm

exog_vars = ['cumsum_reg1','popltn_cascnt']
exog = sm.add_constant(card_cov_pop[exog_vars])
mod = PooledOLS(card_cov_pop["sum"], exog)
pooled_res = mod.fit(cov_type = "robust")
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:                    sum   R-squared:                        0.2048
Estimator:                  PooledOLS   R-squared (Between):              0.5288
No. Observations:                3850   R-squared (Within):               0.0333
Date:                Tue, Dec 15 2020   R-squared (Overall):              0.2048
Time:                        11:53:51   Log-likelihood                -9.633e+04
Cov. Estimator:                Robust                                           
                                        F-statistic:                      495.30
Entities:                          25   P-value                           0.0000
Avg Obs:                       154.00   Distribution:                  F(2,3847)
Min Obs:                       154.00                                           
Max Obs:                       154.00   F-statistic (robust):             78.319
                            

/usr/local/lib/python3.6/dist-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [179]:
# Pooled OLS with cumsum reg1
from linearmodels.panel import PooledOLS
import statsmodels.api as sm

exog_vars = ['cumsum_reg1','popltn_cascnt']
exog = sm.add_constant(card_cov_pop[exog_vars])
mod = PooledOLS(card_cov_pop["mean"], exog)
pooled_res = mod.fit()
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:                   mean   R-squared:                        0.1546
Estimator:                  PooledOLS   R-squared (Between):              0.3175
No. Observations:                3850   R-squared (Within):               0.0663
Date:                Tue, Dec 15 2020   R-squared (Overall):              0.1546
Time:                        11:53:58   Log-likelihood                -6.892e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      351.70
Entities:                          25   P-value                           0.0000
Avg Obs:                       154.00   Distribution:                  F(2,3847)
Min Obs:                       154.00                                           
Max Obs:                       154.00   F-statistic (robust):             351.70
                            

/usr/local/lib/python3.6/dist-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [175]:
# Pooled OLS with cumsum reg1
from linearmodels.panel import PooledOLS
import statsmodels.api as sm

exog_vars = ['count_reg1','popltn_cascnt']
exog = sm.add_constant(card_cov_pop[exog_vars])
mod = PooledOLS(card_cov_pop["mean"], exog)
pooled_res = mod.fit()
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:                   mean   R-squared:                        0.1444
Estimator:                  PooledOLS   R-squared (Between):              0.3104
No. Observations:                3850   R-squared (Within):               0.0544
Date:                Tue, Dec 15 2020   R-squared (Overall):              0.1444
Time:                        11:47:18   Log-likelihood                -6.894e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      324.65
Entities:                          25   P-value                           0.0000
Avg Obs:                       154.00   Distribution:                  F(2,3847)
Min Obs:                       154.00                                           
Max Obs:                       154.00   F-statistic (robust):             324.65
                            

/usr/local/lib/python3.6/dist-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


## Random Effects

In [ ]:
from linearmodels.panel import RandomEffects

exog_vars = ['cumsum_reg1','popltn_cascnt']
mod = RandomEffects(card_cov_pop["mean"], exog)
re_res = mod.fit()
print(re_res)

                        RandomEffects Estimation Summary                        
Dep. Variable:                   mean   R-squared:                        0.0760
Estimator:              RandomEffects   R-squared (Between):              0.1845
No. Observations:                3850   R-squared (Within):               0.0751
Date:                Sat, Dec 12 2020   R-squared (Overall):              0.1136
Time:                        17:58:46   Log-likelihood                -6.827e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      158.15
Entities:                          25   P-value                           0.0000
Avg Obs:                       154.00   Distribution:                  F(2,3847)
Min Obs:                       154.00                                           
Max Obs:                       154.00   F-statistic (robust):             158.15
                            

/usr/local/lib/python3.6/dist-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


## PanelOLS
by entity effects 

In [181]:
from linearmodels.panel import PanelOLS

exog_vars = ['cumsum_reg1','popltn_cascnt']
exog = sm.add_constant(card_cov_pop[exog_vars])
mod = PanelOLS(card_cov_pop["mean"], exog, entity_effects = True)
card_fe_res = mod.fit()
print(card_fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                   mean   R-squared:                        0.0752
Estimator:                   PanelOLS   R-squared (Between):              0.1510
No. Observations:                3850   R-squared (Within):               0.0752
Date:                Tue, Dec 15 2020   R-squared (Overall):              0.1019
Time:                        13:53:47   Log-likelihood                -6.826e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      155.54
Entities:                          25   P-value                           0.0000
Avg Obs:                       154.00   Distribution:                  F(2,3823)
Min Obs:                       154.00                                           
Max Obs:                       154.00   F-statistic (robust):             155.54
                            

/usr/local/lib/python3.6/dist-packages/linearmodels/panel/data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
